# Run train_base.py (LoRA / QLoRA) from Drive
本 Notebook 包含以下步骤：
1. 挂载 Google Drive（读取你已保存的 train_base.py / Configs / Data）
2. 安装依赖（transformers, datasets, peft, bitsandbytes, accelerate, huggingface_hub）
3. 设置 HF_TOKEN（从 Colab Secrets / 交互输入）
4. 做一个小规模 debug 子集并运行 train_base.py 进行快速 smoke-test
5.（可选）运行完整训练

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# 修改为你自己的项目根目录（与 train_base.py 写入位置一致）
BASE_DIR = "/content/drive/MyDrive/AIAA3102/Final_Project"   # <- 如果各位的路径不同，请修改
SCRIPTS_DIR = f"{BASE_DIR}/Scripts"
CONFIGS_DIR = f"{BASE_DIR}/Configs"
DATA_DIR = f"{BASE_DIR}/Data"
MODELS_DIR = f"{BASE_DIR}/Models"

print("BASE_DIR =", BASE_DIR)
!ls -la "{BASE_DIR}"


Mounted at /content/drive
BASE_DIR = /content/drive/MyDrive/AIAA3102/Final_Project
total 296
-rw------- 1 root root  67456 Nov 12 06:36 AIAA3102-FinalProject_Awareness_Ignorance.ipynb
-rw------- 1 root root 112736 Nov 14 16:38 AIAA3102_FinalProject_wyy_01.ipynb
-rw------- 1 root root  68111 Nov 17 11:47 AIAA3102_FinalProject_wyy_02.ipynb
drwx------ 2 root root   4096 Nov  9 10:48 Configs
drwx------ 2 root root   4096 Nov  9 10:48 Data
drwx------ 2 root root   4096 Nov  9 10:50 Deliverables
-rw------- 1 root root  24623 Nov 12 06:00 File_creator.ipynb
drwx------ 2 root root   4096 Nov  9 11:19 .ipynb_checkpoints
drwx------ 2 root root   4096 Nov  9 11:29 Models
drwx------ 2 root root   4096 Nov  9 10:48 Results
drwx------ 2 root root   4096 Nov  9 11:19 Scripts


# Install dependencies and import libraries

In [ ]:
!pip install transformers datasets peft bitsandbytes accelerate huggingface_hub
# 需要一乃乃时间

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 16.2 MB/s eta 0:00:00


# Check Cuda

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
import bitsandbytes as bnb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


# Ensure the configs/script/py files

In [ ]:
import os
paths = {
    "script": f"{SCRIPTS_DIR}/train_base.py",
    "configs": CONFIGS_DIR,
    "train": f"{DATA_DIR}/train.jsonl",
    "valid": f"{DATA_DIR}/valid.jsonl",
    "unknown": f"{DATA_DIR}/unknown_test.jsonl",
}
for k, p in paths.items():
    print(k, "exists:", os.path.exists(p), p)

# 打印前几行检查
print("\n--- train.jsonl (first 3 lines) ---")
!head -n 3 "{paths['train']}"
print("\n--- configs (list) ---")
!ls -la "{CONFIGS_DIR}"


script exists: True /content/drive/MyDrive/AIAA3102/Final_Project/Scripts/train_base.py
configs exists: True /content/drive/MyDrive/AIAA3102/Final_Project/Configs
train exists: True /content/drive/MyDrive/AIAA3102/Final_Project/Data/train.jsonl
valid exists: True /content/drive/MyDrive/AIAA3102/Final_Project/Data/valid.jsonl
unknown exists: True /content/drive/MyDrive/AIAA3102/Final_Project/Data/unknown_test.jsonl

--- train.jsonl (first 3 lines) ---
{"prompt": "Help debug this code snippet by adding comments with the appropriate error messages.\ndef greetings(name):\nnum = 5\nmessage = \"Hello, \" + name + \"!\"\nreturn message", "response": "def greetings(name):\n    # assert name is a string\n    assert isinstance(name, str), \"name must be a string\"\n    num = 5\n    # \"message\" variable should be defined before using it \n    message = \"Hello, \" + name + \"!\"\n    return message"}
{"prompt": "Create a JavaScript function which takes a string and returns the longest word in i

# Debug Dataset Generation

In [ ]:
# Create tiny debug subsets to run a quick smoke-test (avoid long runs)
import shutil
from pathlib import Path
p_data = Path(DATA_DIR)
debug_train = p_data / "train_debug.jsonl"
debug_valid = p_data / "valid_debug.jsonl"

def subset(src, dst, n=20):
    with open(src, 'r', encoding='utf-8') as rf, open(dst, 'w', encoding='utf-8') as wf:
        for i, line in enumerate(rf):
            if i >= n:
                break
            wf.write(line)

subset(paths["train"], debug_train, n=20)
subset(paths["valid"], debug_valid, n=10)
print("Debug subsets created:", debug_train, debug_valid)
!wc -l "{debug_train}" "{debug_valid}"


Debug subsets created: /content/drive/MyDrive/AIAA3102/Final_Project/Data/train_debug.jsonl /content/drive/MyDrive/AIAA3102/Final_Project/Data/valid_debug.jsonl
  20 /content/drive/MyDrive/AIAA3102/Final_Project/Data/train_debug.jsonl
  10 /content/drive/MyDrive/AIAA3102/Final_Project/Data/valid_debug.jsonl
  30 total


**Training**

In [ ]:
# 这个是debug的命令
# 注意：--config_dir 指向你 Drive 下的 Configs 文件夹
'''
!python "{SCRIPTS_DIR}/train_base.py" \
  --config_dir "{CONFIGS_DIR}" \
  --train_file "{DATA_DIR}/train_debug.jsonl" \
  --valid_file "{DATA_DIR}/valid_debug.jsonl" \
  --overwrite_output_dir \
  --num_train_epochs 5 \
  --learning_rate 3e-4 \
  --per_device_train_batch_size 4 \
  --gradient_accumulation_steps 2
  '''

# 这个是正式运行的命令
# 正式训练（按 configs 指定的超参
!python "{SCRIPTS_DIR}/train_base.py" \
  --config_dir "{CONFIGS_DIR}" \
  --train_file "{DATA_DIR}/train.jsonl" \
  --valid_file "{DATA_DIR}/valid.jsonl" \
  --overwrite_output_dir \
  --num_train_epochs 1 \
  --learning_rate 1e-2 \
  --per_device_train_batch_size 32 \
  --metric_for_best_model "eval_loss"

2025-11-17 11:54:15.514191: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763380455.546638    5503 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763380455.556783    5503 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763380455.580552    5503 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763380455.580581    5503 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763380455.580588    5503 computation_placer.cc:177] computation placer alr

In [ ]:
!cp -r /content/models/tinyllama_ai_finetuned_02 /content/drive/MyDrive/AIAA3102/Final_Project/Models

In [ ]:
!rsync -av --delete --progress /content/drive/MyDrive/AIAA3102/Final_Project/Models/tinyllama_ai_finetuned/ /content/models/tinyllama_ai_finetuned/

sending incremental file list
deleting tinyllama_ai_finetuned_02/checkpoint-501/training_args.bin
deleting tinyllama_ai_finetuned_02/checkpoint-501/trainer_state.json
deleting tinyllama_ai_finetuned_02/checkpoint-501/tokenizer_config.json
deleting tinyllama_ai_finetuned_02/checkpoint-501/tokenizer.model
deleting tinyllama_ai_finetuned_02/checkpoint-501/tokenizer.json
deleting tinyllama_ai_finetuned_02/checkpoint-501/special_tokens_map.json
deleting tinyllama_ai_finetuned_02/checkpoint-501/scheduler.pt
deleting tinyllama_ai_finetuned_02/checkpoint-501/scaler.pt
deleting tinyllama_ai_finetuned_02/checkpoint-501/rng_state.pth
deleting tinyllama_ai_finetuned_02/checkpoint-501/optimizer.pt
deleting tinyllama_ai_finetuned_02/checkpoint-501/adapter_model.safetensors
deleting tinyllama_ai_finetuned_02/checkpoint-501/adapter_config.json
deleting tinyllama_ai_finetuned_02/checkpoint-501/README.md
deleting tinyllama_ai_finetuned_02/checkpoint-501/
deleting tinyllama_ai_finetuned_02/checkpoint-400

# Quick Assessment

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import os, json

# 根据 training_args.yaml 的 output_dir 确定保存路径，或改成你想要的 models 路径
import yaml
cfg = yaml.safe_load(open(f"{CONFIGS_DIR}/training_args.yaml"))
outdir = cfg.get("output_dir", f"{BASE_DIR}/models/finetuned_model")
print("Expecting model at:", outdir)

tokenizer = AutoTokenizer.from_pretrained(outdir)
model = AutoModelForCausalLM.from_pretrained(outdir, device_map="auto")
gen = pipeline("text-generation", model=model, tokenizer=tokenizer)

# 测试几条 valid
with open(f"{DATA_DIR}/valid.jsonl",'r',encoding='utf-8') as f:
    lines = [json.loads(l) for l in f][:5]

for ex in lines:
    prompt = f"### 问：\n{ex['prompt']}\n### 答：\n"
    out = gen(prompt, max_new_tokens=128, do_sample=False)[0]["generated_text"]
    print("="*40)
    print("PROMPT:", prompt)
    print("OUTPUT:", out)


Expecting model at: models/tinyllama_ai_finetuned


Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


PROMPT: ### 问：
Create a function in C to check whether a given string contains any punctuations or not.
### 答：

OUTPUT: ### 问：
Create a function in C to check whether a given string contains any punctuations or not.
### 答：

### 答swers

### 答swers

### 答swers

### 答swers

### 答swers

### 答swers

### 答swers

### 答swers

### 答swers

### 答swers

### 答swers

### 答swers

### 答swers

### 答swers

##
PROMPT: ### 问：
Design a function in PHP that takes two strings as input and return true if the strings are an anagram of each other.
string1 = “listen”
string2 = “silent”
### 答：

OUTPUT: ### 问：
Design a function in PHP that takes two strings as input and return true if the strings are an anagram of each other.
string1 = “listen”
string2 = “silent”
### 答：
string1 = string2
string2 = string1
### 答而非
string1 = string2
string2 = string1
### 答而非
string1 = string2
string2 = string1
### 答而非
string1 = string2
string2 = string1
### 答而非
string1 = string2
string2 = string1
### 答而非
string1 = string2
string2 = 

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import os, json
import torch
import yaml
from tqdm import tqdm

class ModelTester:
    def __init__(self, configs_dir, data_dir, base_dir):
        self.configs_dir = configs_dir
        self.data_dir = data_dir
        self.base_dir = base_dir
        self.model = None
        self.tokenizer = None

    def load_model(self):
        """加载模型和tokenizer"""
        # 读取配置
        cfg = yaml.safe_load(open(f"{self.configs_dir}/training_args.yaml"))
        outdir = cfg.get("output_dir", f"{self.base_dir}/models/finetuned_model")

        print(f"🔍 检查模型目录: {outdir}")
        if not os.path.exists(outdir):
            raise FileNotFoundError(f"模型目录不存在: {outdir}")

        # 检查是否是LoRA模型
        try:
            peft_config = PeftConfig.from_pretrained(outdir)
            is_lora = True
            base_model_name = peft_config.base_model_name_or_path
            print(f"✅ 检测到LoRA模型，基础模型: {base_model_name}")
        except:
            is_lora = False
            base_model_name = cfg.get("model_name_or_path", "TinyLlama/TinyLlama_v1.1")
            print(f"✅ 加载完整模型: {base_model_name}")

        # 加载tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(base_model_name)

        # 加载模型
        if is_lora:
            # LoRA模型加载方式
            base_model = AutoModelForCausalLM.from_pretrained(
                base_model_name,
                device_map="auto",
                torch_dtype=torch.float16,
                trust_remote_code=True
            )
            self.model = PeftModel.from_pretrained(base_model, outdir)
            print("✅ LoRA适配器加载完成")
        else:
            # 完整模型加载方式
            self.model = AutoModelForCausalLM.from_pretrained(
                outdir,
                device_map="auto",
                torch_dtype=torch.float16,
                trust_remote_code=True
            )
            print("✅ 完整模型加载完成")

        print(f"📊 模型参数数量: {self.model.num_parameters():,}")
        print(f"💻 模型设备: {self.model.device}")

    def generate_response(self, prompt, generation_config=None):
        """生成回复"""
        if generation_config is None:
            generation_config = {
                "max_new_tokens": 256,
                "do_sample": True,
                "temperature": 0.1,
                "top_p": 0.5,
                "repetition_penalty": 1.1,
                "pad_token_id": self.tokenizer.eos_token_id
            }

        # 编码输入
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)

        # 生成
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                **generation_config
            )

        # 解码输出
        full_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_text = full_text[len(prompt):]

        return full_text, generated_text

    def test_validation_set(self, num_samples=10, save_results=False):
        """测试验证集"""
        print(f"\n🧪 开始验证集测试 ({num_samples}个样本)")

        # 加载验证数据
        with open(f"{self.data_dir}/valid.jsonl", 'r', encoding='utf-8') as f:
            lines = [json.loads(l) for l in f][:num_samples]

        results = []

        for i, ex in enumerate(tqdm(lines, desc="测试进度")):
            prompt = f"问：\n{ex['prompt']}\n答：\n"
            expected_response = ex.get('response', '')

            try:
                full_text, generated_text = self.generate_response(prompt)

                result = {
                    "id": i,
                    "prompt": ex['prompt'],
                    "expected": expected_response,
                    "generated": generated_text,
                    "full_output": full_text
                }
                results.append(result)

                # 打印结果
                print(f"\n{'='*60}")
                print(f"📝 样本 {i+1}:")
                print(f"❓ 问题: {ex['prompt'][:100]}...")
                print(f"🤖 生成回答: {generated_text.strip()}")
                print(f"✅ 期望回答: {expected_response}")

            except Exception as e:
                print(f"❌ 样本 {i+1} 生成失败: {e}")
                continue

        # 保存结果
        if save_results:
            output_file = f"{self.base_dir}/test_results.json"
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(results, f, ensure_ascii=False, indent=2)
            print(f"\n💾 测试结果已保存至: {output_file}")

        return results

    def interactive_mode(self):
        """交互式测试模式"""
        print("\n🎮 进入交互模式 (输入 'quit' 退出)")

        generation_configs = {
            "creative": {
                "max_new_tokens": 300,
                "do_sample": True,
                "temperature": 0.8,
                "top_p": 0.9,
                "top_k": 50
            },
            "precise": {
                "max_new_tokens": 200,
                "do_sample": False,  # 贪婪解码
                "temperature": 1.0,
                "top_p": 1.0
            },
            "balanced": {
                "max_new_tokens": 256,
                "do_sample": True,
                "temperature": 0.7,
                "top_p": 0.9
            }
        }

        current_mode = "balanced"

        while True:
            try:
                user_input = input(f"\n💬 请输入问题 [{current_mode}模式]: ").strip()

                if user_input.lower() in ['quit', 'exit', '退出']:
                    break
                elif user_input.lower() == 'mode':
                    # 切换生成模式
                    print("可用模式: creative, precise, balanced")
                    new_mode = input("选择模式: ").strip()
                    if new_mode in generation_configs:
                        current_mode = new_mode
                        print(f"✅ 切换到 {current_mode} 模式")
                    continue
                elif not user_input:
                    continue

                prompt = f"问：\n{user_input}\n答：\n"
                full_text, generated_text = self.generate_response(
                    prompt,
                    generation_configs[current_mode]
                )

                print(f"\n🤖 模型回答 ({current_mode}模式):")
                print(f"📝 {generated_text.strip()}")

            except KeyboardInterrupt:
                print("\n👋 退出交互模式")
                break
            except Exception as e:
                print(f"❌ 生成失败: {e}")

def main():
    """主函数"""
    tester = ModelTester(CONFIGS_DIR, DATA_DIR, BASE_DIR)

    try:
        # 1. 加载模型
        tester.load_model()

        # 2. 验证集测试
        tester.test_validation_set(num_samples=5, save_results=True)

        # 3. 交互模式
        #tester.interactive_mode()

    except Exception as e:
        print(f"❌ 错误: {e}")

if __name__ == "__main__":
    main()

🔍 检查模型目录: models/tinyllama_ai_finetuned
✅ 检测到LoRA模型，基础模型: TinyLlama/TinyLlama_v1.1
✅ LoRA适配器加载完成
📊 模型参数数量: 1,102,301,184
💻 模型设备: cuda:0

🧪 开始验证集测试 (5个样本)


测试进度:  20%|██        | 1/5 [00:05<00:20,  5.23s/it]


📝 样本 1:
❓ 问题: Create a function in C to check whether a given string contains any punctuations or not....
🤖 生成回答: A: You can use the following code:
#include <stdio.h>
int main() {
    char str[100];
    printf("Enter a string: ");
    scanf("%s", str);
    if (strchr(str, '!') || strchr(str, '#')) {
        printf("The string contains punctuation marks.\n");
    } else {
        printf("The string does not contain punctuation marks.\n");
    }
}
✅ 期望回答: bool containsPunctuation(string s) {
  for (char c : s) {
    if (ispunct(c)) 
      return true;
  }
  return false;
}


测试进度:  40%|████      | 2/5 [00:17<00:28,  9.54s/it]


📝 样本 2:
❓ 问题: Design a function in PHP that takes two strings as input and return true if the strings are an anagr...
🤖 生成回答: string1 = string2
string2 = string1
string1 = string2
string2 = string1
string1 = string2
string2 = string1
string1 = string2
string2 = string1
string1 = string2
string2 = string1
string1 = string2
string2 = string1
string1 = string2
string2 = string1
string1 = string2
string2 = string1
string1 = string2
string2 = string1
string1 = string2
string2 = string1
string1 = string2
string2 = string1
string1 = string2
string2 = string1
string1 = string2
string2 = string1
string1 = string2
string2 = string1
string1 = string2
string2 = string1
string1 = string2
string2 = string1
string1 = string2
string2 = string1
string1 = string2
string2 = string1
string1 = string2
string2 = string1
string1 = string2
string2 = string1
string1 = string2
string2 = string1
string1 = string
✅ 期望回答: function checkAnagram(string1, string2) {
  if(string1.length !== string2.length) {
    retu

...


📝 样本 3:
❓ 问题: Create a custom exception class for a restaurant that has no menu items....
🤖 生成回答: The following code will create an exception class for a restaurant that has no menu items:
class NoMenuException : public Exception {
public:
NoMenuException(const std::string & message) : Exception(message) {}
};
The following code will create an exception class for a restaurant that has one menu item:
class OneMenuItemException : public Exception {
public:
OneMenuItemException(const std::string & message) : Exception(message) {}
};
The following code will create an exception class for a restaurant that has two menu items:
class TwoMenuItemsException : public Exception {
public:
TwoMenuItemsException(const std::string & message) : Exception(message) {}
};
The following code will create an exception class for a restaurant that has three menu items:
class ThreeMenuItemsException : public Exception {
public:
ThreeMenuItemsException(const std::string & message) : Exception(message) {}
};
The

测试进度:  80%|████████  | 4/5 [00:42<00:11, 11.56s/it]


📝 样本 4:
❓ 问题: Using JavaScript, write a function that takes a string as an argument and prints out whether the str...
🤖 生成回答: The function returns true if the string is a palindrome, otherwise it returns false.
function isPalindrome(str) {
var i;
for (i = str.length - 1; i >= 0; i--) {
if (str[i] != str[i + 1]) return false;
return true;
This function is called recursively to check each character in the string.
If the character is not equal to its previous value, then the function returns false. Otherwise, it returns true.
The function is recursive because it calls itself recursively until the string is empty or the length of the string is less than 2.
The function is also written in a way that it can be used for any string.
The function is also written in a way that it can be used for any string. The function is also written in a way that it can be used for any string.
The function is also written in a way that it can be used for any string. The function is also written in a way that

测试进度: 100%|██████████| 5/5 [00:55<00:00, 11.07s/it]


📝 样本 5:
❓ 问题: Create an array of 10 random numbers between 1 and 100 in JavaScript and print each number in the co...
🤖 生成回答: The number of times you have to press the space bar is equal to the number of times you have to press the space bar in JavaScript.
The number of times you have to press the space bar is equal to the number of times you have to press the space bar in JavaScript.
The number of times you have to press the space bar is equal to the number of times you have to press the space bar in JavaScript.
The number of times you have to press the space bar is equal to the number of times you have to press the space bar in JavaScript.
The number of times you have to press the space bar is equal to the number of times you have to press the space bar in JavaScript.
The number of times you have to press the space bar is equal to the number of times you have to press the space bar in JavaScript.
The number of times you have to press the space bar is equal to the number of times you